In [1]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [2]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

/home/svcl-oowl/brandon/research/sil_consistent_at_inference


In [3]:
import pprint
import glob
from pathlib import Path

from tqdm.autonotebook import tqdm
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

from utils import utils

/opt/conda/envs/standard/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [25]:
# this notebook recursively finds all .objs in the input mesh folder and renders them into the output render folder, as jpgs with white backgrounds
# a partition .list file is also generated for the training/validation set

INPUT_MESH_DIR_REAL = "data/misc/example_shapenet"
OUTPUT_RENDER_DIR_REAL = "data/semantic_dis/real_renders"

INPUT_MESH_DIR_FAKE = "data/test_dataset"
OUTPUT_RENDER_DIR_FAKE = "data/semantic_dis/fake_renders"

render_type = "fake"
num_img_to_render = min(len(list(Path(INPUT_MESH_DIR_REAL).rglob('*.obj'))), len(list(Path(INPUT_MESH_DIR_FAKE).rglob('*.obj')))) * 8
print("will render {} {} images".format(num_img_to_render, render_type))
val_split_percentage = 0.25

# render settings
silhouette = True
img_size = 224
device = torch.device("cuda:0")
batch_size = 8
num_azims = 8
# 0.,  45.,  90., 135., 180., 225., 270., 315.
azims = torch.linspace(0, 360, num_azims+1)[:-1]
elevs = torch.ones(num_azims) * 25
dists = torch.ones(num_azims) * 1.7

will render 72 fake images


In [26]:
if render_type == "real":
    OUTPUT_RENDER_DIR = OUTPUT_RENDER_DIR_REAL
    INPUT_MESH_DIR = INPUT_MESH_DIR_REAL
elif render_type == "fake":
    OUTPUT_RENDER_DIR = OUTPUT_RENDER_DIR_FAKE
    INPUT_MESH_DIR = INPUT_MESH_DIR_FAKE
else:
    raise Error("must be real or fake")
    
if not os.path.exists(OUTPUT_RENDER_DIR):
    os.makedirs(OUTPUT_RENDER_DIR)
    
# TODO: randomize?
obj_paths = list(Path(INPUT_MESH_DIR).rglob('*.obj'))

render_i = 0
train_img_names = []
val_img_names = []
with tqdm(total=num_img_to_render) as pbar:
    for model_path in obj_paths:
        with torch.no_grad():
            mesh = utils.load_untextured_mesh(model_path, device)
            renders = utils.batched_render(mesh, azims, elevs, dists, batch_size, device, silhouette, img_size)
            curr_renders_names = []
            for i, render in enumerate(renders):
                
                if silhouette:
                    # turn into hard 0/1 siluette
                    img_render = (render[ ..., 3].cpu().numpy() > 0).astype(int) * 255
                else:
                    img_render = (render[ ..., :3].cpu().numpy()* 255).astype(int) 
                    
                model_name = str(model_path).replace(INPUT_MESH_DIR,'').replace('/','_').replace(".obj","")
                if model_name[0] == '_': model_name = model_name[1:]
                
                render_filename = "{}_{}.jpg".format(model_name, i)
                cv2.imwrite(os.path.join(OUTPUT_RENDER_DIR,render_filename), img_render)
                curr_renders_names.append(render_filename)
                #plt.imshow(img_render)
                #plt.show()
                
                pbar.update(1)
                render_i += 1
            
            if render_i < num_img_to_render * val_split_percentage:
                val_img_names += curr_renders_names
            else:
                train_img_names += curr_renders_names
                
            if render_i >= num_img_to_render:
                break

# creating .lst files to record train/val partition info
with open(os.path.join(OUTPUT_RENDER_DIR, "train.lst"), "w") as f:
    for i, img_name in enumerate(train_img_names):
        if i < len(train_img_names)-1:
            f.write(img_name+"\n")
        else:
            f.write(img_name)

with open(os.path.join(OUTPUT_RENDER_DIR, "val.lst"), "w") as f:
    for i, img_name in enumerate(val_img_names):
        if i < len(val_img_names)-1:
            f.write(img_name+"\n")
        else:
            f.write(img_name)

In [ ]:
# TODO: randomize size?